<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

<h5>Part 1: For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.</h5>
<ol>    
    <li>Start by creating a new Notebook for this assignment.</li>
    <li>Use the Notebook to build the code to scrape the following Wikipedia page, <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">Toronto Postal Codes</a>, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below.</li>
</ol>      

In [ ]:
#install beautiful soup and necessary packages    
!pip install requests beautifulsoup4
!pip install lxml
!pip install pandas lxml

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import lxml
import urllib.request
from bs4 import BeautifulSoup

# location of webpage to scrape for data
online_data_PATH = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [93]:
# scrape website using the beautiful soup package
page = urllib.request.urlopen(online_data_PATH)
soup = BeautifulSoup(page, 'lxml')
all_tables = soup.find_all('table')
right_table = soup.find('table',class_='wikitable sortable')

# build the appropriate data by pulling the relevant cells from the right_table
# the following lists correspond to the series values for each column
# A=Postal Code, B=Borough, C=Neighborhood
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
df = pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


Cleaning the dataframe:

In [95]:
# remove the new line special characters from each entry
postal_code = df["Postal Code"].tolist()
borough = df["Borough"].tolist()
neighborhood = df["Neighborhood"].tolist()

for n in range(len(postal_code)):
    postal_code[n] = postal_code[n].replace("\n","")
    
print(postal_code[0:5])

for n in range(len(borough)):
    borough[n] = borough[n].replace("\n","")
    
print(borough[0:5])

for n in range(len(neighborhood)):
    neighborhood[n] = neighborhood[n].replace("\n","")
    
print(neighborhood[0:5])

['M1A', 'M2A', 'M3A', 'M4A', 'M5A']
['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto']
['', '', 'Parkwoods', 'Victoria Village', 'Regent Park, Harbourfront']


In [96]:
# create a new data frame to clean for the remainder of the project
toronto_df = pd.DataFrame({
    'postal code':postal_code,
    'borough':borough,
    'neighborhood':neighborhood
})

# remove rows with no borough assigned
indexNames = toronto_df[toronto_df['borough'] == 'Not assigned'].index
toronto_df.drop(indexNames, inplace=True)

print(toronto_df.shape)
toronto_df.head()

(103, 3)


,postal code,borough,neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h5>Part 2: Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.</h5>

Get the Coordinates for Each Postal Code

In [97]:
# download the coordinates for each postal code
nbd_coord_df = pd.read_csv('https://cocl.us/Geospatial_data')

# merge coordinates on the postal code column
coord_df = toronto_df.set_index('postal code').join(nbd_coord_df.set_index('Postal Code'))

# reset index and save dataframe
coord_df = coord_df.reset_index()
coord_df.to_csv('./toronto_coord_df.csv')

print(coord_df.shape)
coord_df.head()

(103, 5)


,postal code,borough,neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h5>Part 3: Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.</h5>

In [ ]:
# packages for plotting
!pip install geopy
!pip install folium

In [98]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print('Libraries imported.')

Libraries imported.


Analyze the Initial Data

In [99]:
# create map of Toronto using latitude and longitude values
toronto_latitude = 43.6532
toronto_longitude = -79.3832
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# add markers to map
for lat, lng, postal_code, borough, neighborhood in zip(coord_df['Latitude'], coord_df['Longitude'], coord_df['postal code'], coord_df['borough'], coord_df['neighborhood']):
    label = '{}, {}, {}'.format(postal_code, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Using the Foursquare API

In [100]:
# setup configuration to make foursquare api calls
CLIENT_ID = 'HL3GC4OJTZINFKFIYZEEC535KNHZ3A3DFTQRWCZ34MO2EXUK' # your Foursquare ID
CLIENT_SECRET = 'OURWVENPDO3UAUUFLCYGJLFOM2MRLGAMATSNXTR1WJMQKYEY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HL3GC4OJTZINFKFIYZEEC535KNHZ3A3DFTQRWCZ34MO2EXUK
CLIENT_SECRET:OURWVENPDO3UAUUFLCYGJLFOM2MRLGAMATSNXTR1WJMQKYEY


In [101]:
# function to get the nearby venues for a given postal code
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT=10
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [102]:
# use function above to get top 10 venues near each postal code center
toronto_venues = getNearbyVenues(names=coord_df['postal code'], 
                                 latitudes=coord_df['Latitude'],
                                 longitudes=coord_df['Longitude'])
print(toronto_venues.shape)
toronto_venues.head()

(691, 7)


,Postal Code,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [104]:
# look at how many venues we found for each postal code
toronto_venues.groupby('Postal Code').count().head()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M1B,2,2,2,2,2,2
M1C,2,2,2,2,2,2
M1E,8,8,8,8,8,8
M1G,4,4,4,4,4,4
M1H,8,8,8,8,8,8


In [105]:
# determine the total number of unique values for venue category
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 183 uniques categories.


In [106]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(691, 184)


,Postal Code,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M4A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [107]:
# now group by postal code and take the mean of the frequencies
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(98, 184)


,Postal Code,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [108]:
# print the top 2 venues for each postal code
num_top_venues = 2

for hood in toronto_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                  venue  freq
0  Fast Food Restaurant   0.5
1            Print Shop   0.5


----M1C----
           venue  freq
0  Moving Target   0.5
1            Bar   0.5


----M1E----
                 venue  freq
0        Moving Target  0.12
1  Rental Car Location  0.12


----M1G----
               venue  freq
0        Coffee Shop  0.50
1  Korean Restaurant  0.25


----M1H----
         venue  freq
0         Bank  0.12
1  Gas Station  0.12


----M1J----
                        venue  freq
0  Construction & Landscaping   0.5
1                  Playground   0.5


----M1K----
            venue  freq
0  Discount Store  0.25
1      Hobby Shop  0.12


----M1L----
      venue  freq
0  Bus Line   0.2
1    Bakery   0.2


----M1M----
                 venue  freq
0         Skating Rink  0.33
1  American Restaurant  0.33


----M1N----
          venue  freq
0  Skating Rink   0.2
1          Farm   0.2


----M1P----
                venue  freq
0   Indian Restaurant   0.4
1  Chinese Rest

In [109]:
# sort the top 10 venues for each postal code
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [110]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Print Shop,Discount Store,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,M1C,Bar,Moving Target,Yoga Studio,Distribution Center,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
2,M1E,Breakfast Spot,Moving Target,Electronics Store,Medical Center,Intersection,Mexican Restaurant,Bank,Rental Car Location,Dessert Shop,Drugstore
3,M1G,Coffee Shop,Indian Restaurant,Korean Restaurant,Yoga Studio,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
4,M1H,Bank,Bakery,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Gas Station,Thai Restaurant,Dog Run,Yoga Studio


In [111]:
# cluster the neighborhoods
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [ ]:
# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [114]:
toronto_venues_sorted.head()

,Cluster Labels,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M1B,Fast Food Restaurant,Print Shop,Discount Store,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,4,M1C,Bar,Moving Target,Yoga Studio,Distribution Center,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
2,4,M1E,Breakfast Spot,Moving Target,Electronics Store,Medical Center,Intersection,Mexican Restaurant,Bank,Rental Car Location,Dessert Shop,Drugstore
3,4,M1G,Coffee Shop,Indian Restaurant,Korean Restaurant,Yoga Studio,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
4,4,M1H,Bank,Bakery,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Gas Station,Thai Restaurant,Dog Run,Yoga Studio


In [115]:
toronto_merged = coord_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Postal Code'), on='postal code')

toronto_merged.head() # check the last columns!

,postal code,borough,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Pizza Place,Financial or Legal Service,Coffee Shop,Portuguese Restaurant,Hockey Arena,Yoga Studio,Discount Store,Event Space,Electronics Store,Eastern European Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Breakfast Spot,Historic Site,Coffee Shop,Distribution Center,Restaurant,Spa,Bakery,Park,Gym / Fitness Center,Garden
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,Event Space,Boutique,Vietnamese Restaurant,Gay Bar,Dessert Shop,Electronics Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4.0,Yoga Studio,Arts & Crafts Store,Coffee Shop,Distribution Center,Mexican Restaurant,Burrito Place,Park,Beer Bar,Italian Restaurant,Creperie


In [116]:
# look at unique values for cluster labels to check for errors
print('There are {} unique cluster label.'.format(len(toronto_merged['Cluster Labels'].unique())))
toronto_merged['Cluster Labels'].unique()

There are 6 unique cluster label.


array([ 1.,  2.,  4., nan,  0.,  3.])

In [117]:
# remove any rows not assigned to a cluster
indexNames = toronto_merged[toronto_merged['Cluster Labels'].isnull()].index

# Delete these row indexes from dataFrame
toronto_merged.drop(indexNames, inplace=True)
toronto_merged.head()

,postal code,borough,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Pizza Place,Financial or Legal Service,Coffee Shop,Portuguese Restaurant,Hockey Arena,Yoga Studio,Discount Store,Event Space,Electronics Store,Eastern European Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Breakfast Spot,Historic Site,Coffee Shop,Distribution Center,Restaurant,Spa,Bakery,Park,Gym / Fitness Center,Garden
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,Event Space,Boutique,Vietnamese Restaurant,Gay Bar,Dessert Shop,Electronics Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4.0,Yoga Studio,Arts & Crafts Store,Coffee Shop,Distribution Center,Mexican Restaurant,Burrito Place,Park,Beer Bar,Italian Restaurant,Creperie


In [118]:
# change data type of cluster to int
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int64')

print(toronto_merged.dtypes)
toronto_merged.head()

postal code                object
borough                    object
neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object


,postal code,borough,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Park,Food & Drink Shop,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Pizza Place,Financial or Legal Service,Coffee Shop,Portuguese Restaurant,Hockey Arena,Yoga Studio,Discount Store,Event Space,Electronics Store,Eastern European Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,Breakfast Spot,Historic Site,Coffee Shop,Distribution Center,Restaurant,Spa,Bakery,Park,Gym / Fitness Center,Garden
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,Event Space,Boutique,Vietnamese Restaurant,Gay Bar,Dessert Shop,Electronics Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Yoga Studio,Arts & Crafts Store,Coffee Shop,Distribution Center,Mexican Restaurant,Burrito Place,Park,Beer Bar,Italian Restaurant,Creperie


In [119]:
# visualize our clusters
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['postal code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# consider the theme and label each cluster
# look at cluster 1
toronto_baseball_cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

# look at cluster 2
toronto_park_cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

# look at cluster 3
toronto_pizza_cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

# look at cluster 4
toronto_garden_cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

# look at cluster 5
toronto_fitness_cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


In [121]:
# cluster 1
toronto_baseball_cluster.head()

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,0,Korean Restaurant,Baseball Field,Food Truck,Business Service,Yoga Studio,Donut Shop,Farm,Falafel Restaurant,Event Space,Electronics Store
57,North York,0,Baseball Field,Yoga Studio,Distribution Center,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
101,Etobicoke,0,Construction & Landscaping,Baseball Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore


In [122]:
# cluster 2
toronto_park_cluster.head()

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Park,Food & Drink Shop,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
10,North York,1,Park,Japanese Restaurant,Sushi Restaurant,Pub,Discount Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
21,York,1,Park,Women's Store,Pool,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
35,East York,1,Park,Convenience Store,Metro Station,Discount Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
40,North York,1,Park,Airport,Bus Stop,Business Service,Snack Place,Distribution Center,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant


In [123]:
# cluster 3
toronto_pizza_cluster.head()

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,Pizza Place,Financial or Legal Service,Coffee Shop,Portuguese Restaurant,Hockey Arena,Yoga Studio,Discount Store,Event Space,Electronics Store,Eastern European Restaurant
6,Scarborough,2,Fast Food Restaurant,Print Shop,Discount Store,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
8,East York,2,Pizza Place,Pharmacy,Fast Food Restaurant,Gym / Fitness Center,Gastropub,Athletics & Sports,Café,Intersection,Bank,Distribution Center
29,East York,2,Indian Restaurant,Yoga Studio,Sandwich Place,Burger Joint,Coffee Shop,Grocery Store,Pharmacy,Gym,Warehouse Store,Toy / Game Store
46,North York,2,Grocery Store,Hotel,Shopping Mall,Bank,Park,Gastropub,Discount Store,Event Space,Gift Shop,Electronics Store


In [124]:
# cluster 4
toronto_garden_cluster.head()

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,3,Garden,Home Service,Yoga Studio,Discount Store,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore


In [125]:
# cluster 5
toronto_fitness_cluster.head()

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4,Breakfast Spot,Historic Site,Coffee Shop,Distribution Center,Restaurant,Spa,Bakery,Park,Gym / Fitness Center,Garden
3,North York,4,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,Event Space,Boutique,Vietnamese Restaurant,Gay Bar,Dessert Shop,Electronics Store
4,Downtown Toronto,4,Yoga Studio,Arts & Crafts Store,Coffee Shop,Distribution Center,Mexican Restaurant,Burrito Place,Park,Beer Bar,Italian Restaurant,Creperie
7,North York,4,Gym / Fitness Center,Japanese Restaurant,Athletics & Sports,Café,Caribbean Restaurant,Yoga Studio,Dog Run,Farm,Falafel Restaurant,Event Space
9,Downtown Toronto,4,Café,Tea Room,Comic Shop,Music Venue,Burrito Place,Clothing Store,Plaza,Theater,Thai Restaurant,Electronics Store
